In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [41]:
with open('../data/headers_mod.csv') as f:
    mod_headers = f.readlines()[0].split()
    
with open('../data/headers_obs.csv') as f:
    obs_headers = f.readlines()[0].split()
    
mod_headers, obs_headers

In [42]:
targets = pd.read_csv('../data/targets.csv')
stations = pd.read_csv('../data/stations.csv')

In [43]:
targets.head()

,date,station,target
0,2015-01-03,ES0691A,0
1,2015-01-03,ES1396A,0
2,2015-01-03,ES1438A,0
3,2015-01-03,ES1480A,0
4,2015-01-03,ES1679A,0


In [44]:
stations.head()

,code,name,lat,lon,height,city,country
0,ES1480A,Barcelona (Gràcia-Sant Gervasi),41.3987,2.15339,57,Barcelona,España
1,ES1856A,Barcelona (Vall d'Hebron),41.4261,2.14799,136,Barcelona,España
2,ES1679A,Ciutadella,41.3864,2.18742,7,Barcelona,España
3,ES1438A,Barcelona (Eixample),41.3853,2.15382,26,Barcelona,España
4,ES1396A,Barcelona,41.3788,2.13310,35,Barcelona,España
